# Capstone Project - Car accident severity  (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

Because of the increase in the number of cars and the raise of accidents happened in the roads too, this cause injuries or fatalities to people, waste time and cost for police and rescue vehicles, traffic delay in the area.

The Seattle Department of Transportation (SODT) objective to improve new model aims to predict the severity of car accidents and main variables that strongly related to it, in order to help decisions based to make campaigns or improvements to decrease severe accidents based on certain conditions. And detect intersection have car accidents with hight servity .

## Data <a name="data"></a>

**Data Understanding**

In this phase, and based on definition of our problem, many factors that will influence our decission . I will use csv file provided by Coursera. The main attributes I will use as independent variables are: WEATHER, ROADCOND and LIGHTCOND, that adequate to train my machine learning model. Also, I will assess the condition of these attributes by looking for skewed information and the correlations between them and the severity of the accident, so the "SEVERITYCODE" used as labeld data  property damage (class 1) or injury (class 2) .

**Data preparation**

In this phase I will build the final dataset that fed my model by:

    *Filling missing data
    * Decodes categoral data
    *Finally, balance and normalize all data.

**Methodolegy**

After the data prepeared and processed i will splitt it by 0.3.

We will use the following models:

        K-Nearest Neighbor (KNN)
        KNN will help us predict the severity code of an outcome by finding the most similar to data point within k distance.

        Decision Tree
        A decision tree model gives us a layout of all possible outcomes so we can fully analyze the concequences of a decision. It context, the decision tree observes all possible outcomes of different weather conditions.

        Logistic Regression
        Because our dataset only provides us with two severity code outcomes, our model will only predict one of those two classes. This makes our data binary, which is perfect to use with logistic regression.
Then make evaluation on test data .

**Analysis**

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn import metrics

from sklearn.metrics import classification_report, confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [2]:
df = pd.read_csv("Data-Collisions.csv")

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
dataset = df.drop(columns = ["SEVERITYCODE.1","X","Y","OBJECTID","INCKEY","COLDETKEY","REPORTNO","STATUS","ADDRTYPE","INTKEY","LOCATION","EXCEPTRSNCODE","EXCEPTRSNDESC","SEVERITYDESC","COLLISIONTYPE","PERSONCOUNT","PEDCOUNT","PEDCYLCOUNT","VEHCOUNT","INCDATE","INCDTTM","JUNCTIONTYPE","SDOT_COLCODE","SDOT_COLDESC","INATTENTIONIND","UNDERINFL","PEDROWNOTGRNT","SDOTCOLNUM","SPEEDING","ST_COLCODE","ST_COLDESC","SEGLANEKEY","CROSSWALKKEY","HITPARKEDCAR"] )
dataset.head ()

,SEVERITYCODE,WEATHER,ROADCOND,LIGHTCOND
0,2,Overcast,Wet,Daylight
1,1,Raining,Wet,Dark - Street Lights On
2,1,Overcast,Dry,Daylight
3,1,Clear,Dry,Daylight
4,2,Raining,Wet,Daylight


Find Missing Data and Uniques Values 

In [4]:
dataset.nunique()

SEVERITYCODE     2
WEATHER         11
ROADCOND         9
LIGHTCOND        9
dtype: int64

In [5]:
dataset.isna().sum()

SEVERITYCODE       0
WEATHER         5081
ROADCOND        5012
LIGHTCOND       5170
dtype: int64

Change categories  to Numbers 

In [6]:
dataset["WEATHER"] = dataset["WEATHER"].astype('category')
dataset["ROADCOND"] = dataset["ROADCOND"].astype('category')
dataset["LIGHTCOND"] = dataset["LIGHTCOND"].astype('category')

dataset["WEATHER_CAT"] = dataset["WEATHER"].cat.codes
dataset["ROADCOND_CAT"] = dataset["ROADCOND"].cat.codes
dataset["LIGHTCOND_CAT"] = dataset["LIGHTCOND"].cat.codes


Data Balance

In [7]:
dataset.groupby(["SEVERITYCODE"]).count()

,WEATHER,ROADCOND,LIGHTCOND,WEATHER_CAT,ROADCOND_CAT,LIGHTCOND_CAT
SEVERITYCODE,,,,,,
1,132488,132533,132405,136485,136485,136485
2,57104,57128,57098,58188,58188,58188


In [8]:
from sklearn.utils import resample 

dataset_1 = dataset[dataset.SEVERITYCODE==1]
dataset_2 = dataset[dataset.SEVERITYCODE==2]

datset_1_sample = resample(dataset_1,replace = False ,n_samples=58188,random_state=99)

datset_balanced =  pd.concat( [datset_1_sample,dataset_2] )


Split data to Training and Testing

In [9]:
import numpy as np 

X = np.asarray(datset_balanced[["WEATHER_CAT","ROADCOND_CAT","LIGHTCOND_CAT"]])
y = np.asarray(datset_balanced["SEVERITYCODE"])

from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

from sklearn.model_selection import train_test_split
X_train, X_test ,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=3)


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


## Analysis by Model <a name="analysis"></a>

Let's perform some models (Logistc,)

**1) Using Logistic Regression Model**

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

LRBest = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)


**2) KNN**

In [11]:
from sklearn.neighbors import KNeighborsClassifier
Ks = 10
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
ConfustionMx = [];
for n in range(1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    yhat=neigh.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)  
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])
KNNBest = KNeighborsClassifier(n_neighbors = mean_acc.argmax()+1).fit(X_train,y_train)

**3)  DecisionTree**

In [12]:
from sklearn.tree import DecisionTreeClassifier
Ks = 10
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
ConfustionMx = [];
for n in range(1,Ks):
    
    #Train Model and Predict  
    
    DTree = DecisionTreeClassifier(criterion="entropy", max_depth = n)
    DTree.fit(X_train,y_train)
    yhat=DTree.predict(X_test)
    
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])

DTreeBest = DecisionTreeClassifier(criterion="entropy", max_depth = mean_acc.argmax()+1).fit(X_train,y_train)

## Model Evaluation using Test set 

In [19]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

yhat = KNNBest.predict(X_test) 
KNNJss = '{:0.2f}'.format(jaccard_similarity_score(y_test, yhat))
KNNF1 =  '{:0.2f}'.format(f1_score(y_test, yhat, average='weighted'))

yhat = DTreeBest.predict(X_test) 
DTJss =  '{:0.2f}'.format(jaccard_similarity_score(y_test, yhat))
DTF1 = '{:0.2f}'.format(f1_score(y_test, yhat, average='weighted'))


yhat = LRBest.predict(X_test)
yhatprob = LRBest.predict_proba(X_test)
LRJss =  '{:0.2f}'.format(jaccard_similarity_score(y_test, yhat))
LRF1 = '{:0.2f}'.format(f1_score(y_test, yhat, average='weighted'))
LRlog = '{:0.2f}'.format(log_loss(y_test, yhatprob))

data = np.array([['Algorithm','Jaccard', 'F1-score', 'Logloss'],
                ['KNN',KNNJss,KNNF1,'NA'],
                ['Decision Tree',DTJss,DTF1,'NA'],
               # ['SVM',SVMJss,SVMF1,'NA'],
                ['Logistic Regression',LRJss,LRF1,LRlog]
                
                ])

Reportdf = pd.DataFrame(data=data[1:,1:],
                  index=data[1:,0],
                  columns=data[0,1:])
Reportdf.columns.name = 'Algorithm'

print(Reportdf   )

Algorithm           Jaccard F1-score Logloss
KNN                    0.56     0.54      NA
Decision Tree          0.56     0.49      NA
Logistic Regression    0.53     0.51    0.68


## Results and Discussion <a name="results"></a>

In the beginning of this notebook, we had categorical data that need to encoded becuase they not a data type that we could have fed through an algoritim, so label encoding was used to created new classes that were of type int8; a numerical data type.

After that issue we were presented with another  imbalanced of data. As mentioned earlier, class 1 was nearly three times larger than class 2, So the solution to this was resampling the majority class to 58188 values each.

Once we analyzed and cleaned the data, it was then fed through three ML models; K-Nearest Neighbor, Decision Tree and Logistic Regression. Although the first two are ideal for this project, logistic regression made most sense because of its binary nature.

Evaluation metrics used to test the accuracy of our models were jaccard index, f-1 score and logloss for logistic regression.



## Conclusion <a name="conclusion"></a>

Based on historical data from weather conditions pointing to certain classes, we can conclude that particular weather conditions have a somewhat impact on whether or not travel could result in property damage (class 1) or injury (class 2).